In [2]:
print("Hello, world")

Hello, world


In [3]:
import pandas as pd
import numpy as np
import faiss

from tqdm.notebook import tqdm
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MaxAbsScaler

In [4]:
important_feautres = ['20', 'norm', 'log_20', '63_minus_10', '10_minus_63', '63', '10',
       '63_minus_35', 'log_33', '35_plus_63', '46', '64', '61', '49',
       '55', '35', '12', '8', '36', '39', '24', '26', '29', '60', '32',
       '28', '27', '15', '52', '19', '37', '68', '22', '67', '3', '66',
       '47', '30', '18', '56', '4', '50', '48', '11', '45', '58', '62',
       '9', '34', '0', '41', '14', '17', '31', '71', '69', '43', '57',
       '40', '53', '51', '16', '38']

## Head-on Solution

In [5]:
df_base = pd.read_parquet("../tmp/cleared_df_base_new_features.parquet")

In [6]:
df_base.head()

,0,1,2,3,4,5,7,8,9,10,...,59,65,70,63_minus_10,10_minus_63,63_minus_35,35_plus_63,log_20,log_33,norm
Id,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083893,11.152912,-64.426758,-118.880890,216.482437,-104.698059,44.348083,120.915344,181.449707,-124.061508,...,1.541230,-0.722587,1.349423,162.862335,-162.862335,72.717850,150.319504,10803.333984,0.021330,10833.963867
1-base,-34.562202,13.332763,-69.787613,-166.533478,57.680607,-86.098373,-35.637436,119.718636,195.234192,-141.304352,...,0.212176,-0.918577,1.424935,182.404358,-182.404358,12.294678,94.494675,6689.283691,0.740848,6740.816895
2-base,-54.233746,6.379371,-29.210136,-133.413834,150.895828,-99.435326,62.381706,128.951447,164.381470,-140.822449,...,-0.820666,0.941837,-0.520661,186.834259,-186.834259,33.769753,125.793358,6798.427246,0.516188,6843.204102
3-base,-87.520126,4.037884,-87.803032,-185.067627,76.369537,-58.985165,-33.611237,122.031914,136.233582,-108.470444,...,-0.865241,-0.405384,-0.520661,102.111526,-102.111526,99.165222,86.447388,11651.750977,0.000568,11676.433594
4-base,-72.743851,6.522048,43.671265,-140.608032,5.820023,-112.074081,45.182499,122.167183,112.119064,-110.342453,...,-0.822464,0.541299,-0.520661,166.984863,-166.984863,14.419121,127.703926,3345.922852,0.102511,3439.661621


In [7]:
df_train = pd.read_parquet("../tmp/cleared_df_train_new_features.parquet")
df_train.head()

,0,1,2,3,4,5,7,8,9,10,...,71,6,21,25,33,44,59,65,70,norm
Id,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.936676,187.517487,-87.144928,38.307602,109.085564,30.413513,-88.082687,...,-25.958572,0.169541,-0.830982,0.247134,0.860725,0.395763,1.954331,-1.009224,0.778056,830.638733
1-query,-87.776367,6.806268,-32.054546,-177.260391,120.803329,-83.810593,-78.433090,124.915901,140.331070,-177.605804,...,-51.276833,0.879729,-1.554872,0.247134,1.176890,1.297239,-0.523412,-0.117699,0.069525,759.059692
2-query,-49.979565,3.841486,-116.118591,-180.401978,190.128433,-50.837620,-30.447489,125.771164,211.607819,-86.346558,...,-22.547178,1.416623,0.503497,0.247134,-1.887753,0.471004,0.555915,0.513739,-0.520661,809.018555
3-query,-47.810562,9.086598,-115.401695,-121.011360,94.652840,-109.255409,79.186523,124.003098,242.650650,-146.517075,...,-19.902788,-2.195064,0.503497,0.247134,-1.887753,0.447305,-0.227860,0.609955,-0.520661,779.801270
4-query,-79.632126,14.442886,-58.903397,-147.052536,57.127068,-16.239529,45.984676,125.941284,103.392670,-107.153023,...,-21.149351,0.233082,0.503497,0.247134,-0.718462,-0.680013,0.396024,-1.228321,-0.520661,729.090942


In [8]:
dims = df_base.shape[1]
n_cells = 50
# how many cells
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [9]:
idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))

In [10]:
base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
#del df_base

In [11]:
headon_solution_df_train = df_train.copy()
targets = headon_solution_df_train["Target"]
headon_solution_df_train.drop("Target", axis=1, inplace=True)

In [12]:
%%time
k=5
vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)

acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(idx))

60.551
CPU times: total: 10min 16s
Wall time: 28.5 s


### Вывод
Решение в лоб - 5.575 попугаев.
Очень низкая оценка.

## Scalers

### StandardScaler

In [13]:
def checkStandardScaler(base, train):
    scaler = StandardScaler()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [14]:
sscaler_base, sscaler_train, _ = checkStandardScaler(df_base, df_train)

In [15]:
def buildA5Score(df_base, df_train, short=True, k=10):
    dims = df_base.shape[1]
    n_cells = 50
    # how many cells
    quantizer = faiss.IndexFlatL2(dims)
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
    idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
    idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] for r in el])
    
    return 100 * acc / len(idx)

In [16]:
%%time
print("A@5 Score on Standard Scaler :", buildA5Score(sscaler_base, sscaler_train))

A@5 Score on Standard Scaler : 59.031
CPU times: total: 11min 46s
Wall time: 33.7 s


### RobustScaler

In [8]:
def checkRobustScaler(base, train):
    scaler = RobustScaler()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [18]:
important_feautres_for_train = important_feautres.copy()
important_feautres_for_train.append("Target")
rscaler_base, rscaler_train, _ = checkRobustScaler(df_base[important_feautres], df_train[important_feautres_for_train])

In [19]:
%%time
print("A@5 Score on Robust Scaler :", buildA5Score(rscaler_base, rscaler_train, k=5))

A@5 Score on Robust Scaler : 61.621
CPU times: total: 10min 30s
Wall time: 29.2 s


### MinMaxScaler

In [20]:
def checkMinMaxScaler(base, train):
    scaler = MinMaxScaler()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [21]:
mmscaler_base, mmscaler_train, _ = checkMinMaxScaler(df_base, df_train)

In [22]:
%%time
print("A@5 Score on Min Max Scaler :", buildA5Score(mmscaler_base, mmscaler_train))

A@5 Score on Min Max Scaler : 61.7
CPU times: total: 11min 58s
Wall time: 34.4 s


### Normalizer

In [23]:
def checkNormalizer(base, train):
    scaler = Normalizer()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [24]:
normscaler_base, normscaler_train, _ = checkNormalizer(df_base, df_train)

In [25]:
%%time
print("A@5 Score on Normalizer :", buildA5Score(normscaler_base, normscaler_train))

A@5 Score on Normalizer : 62.008
CPU times: total: 10min 36s
Wall time: 30.5 s


### MaxAbsScaler

In [26]:
def checkMaxAbsScaler(base, train):
    scaler = MaxAbsScaler()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [27]:
maxAbsscaler_base, maxAbsscaler_train, _ = checkMaxAbsScaler(df_base, df_train)

In [28]:
%%time
print("A@5 Score on Max Abs Scaler :", buildA5Score(maxAbsscaler_base, maxAbsscaler_train))

A@5 Score on Max Abs Scaler : 50.261
CPU times: total: 11min 42s
Wall time: 33.7 s


## FIASS Tuning

### IndexLSH

In [10]:
lsh_failed_targets = []

def buildA5ScoreV2(df_base, df_train, short=True, k_value=50):
    dims = df_base.shape[1]
    n_cells = 100
    # how many cells
    #quantizer = faiss.IndexHNSWFlat(dims)
    #idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
    idx_l2 = faiss.IndexLSH(dims, 2048)
    if short:
        idx_l2.train(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    else:
        idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    k=k_value
    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        plus_one = int(target in [base_index[r] for r in el])
        acc += plus_one
        if plus_one == 0:
            lsh_failed_targets.append(target)
    
    return 100 * acc / len(idx), idx_l2

In [11]:
important_feautres = ['20', 'norm', 'log_20', '63_minus_10', '10_minus_63', '63', '10',
       '63_minus_35', 'log_33', '35_plus_63', '46', '64', '61', '49',
       '55', '35', '12', '8', '36', '39', '24', '26', '29', '60', '32',
       '28', '27', '15', '52', '19', '37', '68', '22', '67', '3', '66',
       '47', '30', '18', '56', '4', '50', '48', '11', '45', '58', '62',
       '9', '34', '0', '41', '14', '17', '31', '71', '69', '43', '57',
       '40', '53', '51', '16', '38']

important_feautres_for_train = important_feautres.copy()
important_feautres_for_train.append("Target")

In [139]:
#numer_of_imprtant_features = len(important_feautres)
#mat = faiss.PCAMatrix (numer_of_imprtant_features, numer_of_imprtant_features-10)
#mat.train(df_base[important_feautres].values)
#assert mat.is_trained
#base_array = mat.apply(df_base[important_feautres].values)
#train_array = mat.apply(df_train[important_feautres].values)
# print this to show that the magnitude of tr's columns is decreasin
#df_pca_base = pd.DataFrame(base_array)
#df_pca_train = pd.DataFrame(train_array)
#df_pca_train["Target"] = pd.Series(df_train["Target"].values)


In [12]:
rscaler_base, rscaler_train, _ = checkRobustScaler(df_base[important_feautres], df_train[important_feautres_for_train])

KeyError: "['log_20', '63_minus_10', '10_minus_63', '63_minus_35', 'log_33', '35_plus_63', 'Target'] not in index"

In [ ]:
%%time
lsh_score, lsh_index = buildA5ScoreV2(rscaler_base, rscaler_train, short=False, k_value=5)
print("A@5 Score on Robust Scaler V2:", lsh_score)

In [150]:
lsh_score

69.675

In [44]:
lsh_failed_targets[0]

'675816-base'

In [62]:
train1 = rscaler_train[rscaler_train['Target'] == '675816-base']

In [71]:
base1 = rscaler_base[rscaler_base.index == '675816-base']
base1

,46,67,37,62,32,26,27,47,31,61,...,41,7,56,35,13,11,1,54,42,21
Id,,,,,,,,,,,,,,,,,,,,,
675816-base,-0.285559,0.304065,-0.249466,-0.715156,0.542517,-0.32499,-0.396894,-0.954194,-0.27643,-0.415762,...,-0.130911,-0.087255,0.688407,-1.084891,0.101657,0.328388,1.349459,-0.554402,-0.210944,0.0


In [72]:
base2 = rscaler_base[rscaler_base.index == '755584-base']
base2

,46,67,37,62,32,26,27,47,31,61,...,41,7,56,35,13,11,1,54,42,21
Id,,,,,,,,,,,,,,,,,,,,,
755584-base,-1.158948,-0.44583,1.202662,0.964724,-1.022237,-0.233069,-0.446255,0.039741,-0.740189,0.108479,...,-0.678469,-1.508571,-1.330712,0.204835,-0.541061,-1.432634,-0.260633,-0.82335,0.846664,-0.689275


In [65]:
test1 = train1.copy()
test1.pop("Target")
np.linalg.norm(test1.values)

6.265501

In [73]:
test2 = base1.copy()
np.linalg.norm(test2.values)

6.282187

In [74]:
test3 = base2.copy()
np.linalg.norm(test3.values)

5.7175574

In [70]:
#search_data = rscaler_train[rscaler_train['Target'] == '675816-base'];
vecs, idx = lsh_index.search(np.ascontiguousarray(test1.values).astype('float32'), 100)
idx

array([[ 755584, 1136231,  143978,  336969,  232405,  454434, 1113711,
        2360257, 1818641,   89840,  598613,  142730, 1037661, 1299775,
         352715,  792610, 1307670,  629775,  455829, 1653095,  845854,
         233844, 2295048,  223859,  480296, 2479662, 1394044,  271665,
          13374,  960901, 1375244,  583287,  816981, 2054647,  251039,
          58509, 1674977, 1472986,   57113,  450667, 1934845, 2700063,
        2346335,  682218,   49353, 2356601, 1646333, 1142019, 1631947,
        2186758, 1159463, 1374819, 1598432, 1747988,  622880,  854264,
         445520,  382333, 1324342,  547719,  728097, 1121417, 1679537,
        2212973, 2398299,  323974,  363988, 2711635, 2882082,  629636,
        1224144, 1639495,  443428,  105660,  851029, 1713683, 1160145,
        2042379,  725525, 1546520,  485661,  679629,  200263,  669932,
         751217,  988777, 2145555,  703484, 2909593, 1162417, 1409261,
         610835, 1307193, 2531948, 1379669, 1777975,  458465,  986050,
      

### IndexHNSWFlat

In [103]:
def buildA5ScoreV3(df_base, df_train, short=True, k_value=5):
    dims = df_base.shape[1]
    n_cells = 100
    # how many cells
    #quantizer = faiss.IndexFlatL2(dims)
    #idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
    #idx_l2 = faiss.IndexLSH(dims, 1024)
    #M = 2
    #d2 = int((dims + M - 1) / M) * M
    #q = faiss.IndexPQ (d2, M, 4)
    #index_pq = faiss.IndexRefineFlat(q)
    #remapper = faiss.RemapDimensionsTransform (dims, d2, True)
    #idx_l2 = faiss.IndexPreTransform (remapper, index_pq)
    #idx_l2 = faiss.index_factory(dims, "OPQ16_64,IMI2x8,PQ8+16")

    idx_l2 = index = faiss.IndexHNSWFlat(dims, 512, faiss.METRIC_JensenShannon)
    
    if short:
        idx_l2.train(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    else :
        idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    k=k_value
    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] if r in base_index else 0 for r in el])
    
    return 100 * acc / len(idx)



In [ ]:
%%time
print("A@5 Score on Robust Scaler V3:", buildA5ScoreV3(rscaler_base, rscaler_train, short=False, k_value=5))

In [38]:
def buildA5ScoreV4(df_base, df_train, short=True, k_value=5):
    dims = df_base.shape[1]
    n_cells = 100
    # how many cells
    quantizer = faiss.IndexFlatL2(dims)
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

    if short:
        idx_l2.train(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    else :
        idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    k=5
    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] if r in base_index else 0 for r in el])
    
    return 100 * acc / len(idx)
    

In [35]:
%%time
print("A@5 Score on Robust Scaler V3:", buildA5ScoreV4(rscaler_base, rscaler_train, short=False, k_value=5))

A@5 Score on Robust Scaler V3: 70.065
CPU times: total: 5min 42s
Wall time: 25.2 s


In [36]:
def buildA5ScoreV5(df_base, df_train, short=True, k_value=5):
    dims = df_base.shape[1]
    n_cells = 100
    # how many cells
    idx_l2 = faiss.index_factory(dims, "IVF69,SQ8")


    if short:
        idx_l2.train(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    else :
        idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))

    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    k=k_value
    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] if r in base_index else 0 for r in el])
    
    return 100 * acc / len(idx)
    

In [37]:
%%time
print("A@5 Score on Robust Scaler V3:", buildA5ScoreV5(rscaler_base, rscaler_train, short=False))

A@5 Score on Robust Scaler V3: 63.009
CPU times: total: 8min 6s
Wall time: 22.8 s
